# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [29]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [34]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [37]:
%%capture --no-display

# Configure the map plot
plot_1 = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True, 
    tiles = 'OSM', 
    frame_width = 700, 
    frame_height = 500, 
    color='City',
    alpha= 0.5,
    s='Humidity',
    scale= 0.8)


# Display the map
plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [40]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_df = city_data_df.loc[(city_data_df['Max Temp']<=12)\
                            &(city_data_df['Max Temp']>=0)\
                            &(city_data_df['Cloudiness']<=90)\
                            &(city_data_df['Country']!='US')\
                            &(city_data_df['Country']!='AU')\
                            &(city_data_df['Country']!='IN')\
                            &(city_data_df['Country']!='RU')\
                            &(city_data_df['Country']!='CN')\
                            &(city_data_df['Humidity']<70),:]
print(len(ideal_df))

# Drop any rows with null values
ideal_df.dropna()

# Display sample data
ideal_df.head()

9


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
171,171,la ronge,55.1001,-105.2842,5.14,52,75,2.57,CA,1666108339
222,222,hay river,60.8156,-115.7999,10.99,58,40,3.60,CA,1666108369
250,250,floro,61.5996,5.0328,9.04,61,40,6.69,NO,1666108384
262,262,thompson,55.7435,-97.8558,1.09,60,40,3.60,CA,1666108392
331,331,swan river,52.1058,-101.2676,0.94,40,0,2.06,CA,1666108431


### Step 3: Create a new DataFrame called `hotel_df`.

In [43]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_df.iloc[:,[1,8,2,3,5]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']=''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
171,la ronge,CA,55.1001,-105.2842,52,
222,hay river,CA,60.8156,-115.7999,58,
250,floro,NO,61.5996,5.0328,61,
262,thompson,CA,55.7435,-97.8558,60,
331,swan river,CA,52.1058,-101.2676,40,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [46]:
# Set parameters to search for a hotel
radius = 10000
params = {'categories':'accommodation.hotel',\
         'filter':'',\
         'bias':'',\
         'apiKey':geoapify_key,\
         'limit':1}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lon = row['Lng']
    lat = row['Lat']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lon},{lat},{radius}'
    params["bias"] = f'proximity:{lon},{lat}'

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
la ronge - nearest hotel: La Ronge Hotel and Suites
hay river - nearest hotel: No hotel found
floro - nearest hotel: Comfort Hotel Victoria Florø
thompson - nearest hotel: Thompson Inn
swan river - nearest hotel: Westwood Inn
aktau - nearest hotel: Гульдана
mandalgovi - nearest hotel: Б
bulgan - nearest hotel: Bulgan Hotel
sorland - nearest hotel: Fagmek Resorts


,City,Country,Lat,Lng,Humidity,Hotel Name
171,la ronge,CA,55.1001,-105.2842,52,La Ronge Hotel and Suites
222,hay river,CA,60.8156,-115.7999,58,No hotel found
250,floro,NO,61.5996,5.0328,61,Comfort Hotel Victoria Florø
262,thompson,CA,55.7435,-97.8558,60,Thompson Inn
331,swan river,CA,52.1058,-101.2676,40,Westwood Inn
428,aktau,KZ,43.6500,51.2000,46,Гульдана
532,mandalgovi,MN,45.7625,106.2708,23,Б
555,bulgan,MN,48.8125,103.5347,40,Bulgan Hotel
556,sorland,NO,67.6670,12.6934,67,Fagmek Resorts


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [49]:
%%capture --no-display

# Configure the map plot
plot_4 = hotel_df.hvplot.points('Lng',
            'Lat',
            geo=True, 
            tiles = 'OSM', 
            frame_width = 700, 
            frame_height = 500, 
            color='City',
            alpha=0.6,
            hover=True,
            hover_cols='all',
            use_index=False,
            s='Humidity',
            scale= 1.2)

# Display the map
plot_4

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)